In [4]:
import pandas as pd

In [ ]:
# there are ratios in the data, can they be used?
# (I dont think interpretations alone are going to cut it)
# So what can I do with these ratios?

In [ ]:
# ooh, if I can find something like LCR(ofc, excepting other ratios such as quick), then I can generate a 
# basic basel report> bool:compliant/not
# you dont really need uslocs for this cos you dont need large time data, you are calculating assets for a point
# so, use balance sheets

In [226]:
"""I can do simple metrics calculaitons and maybe predict them, but the regression does not seem to be working"""

'I can do simple metrics calculaitons and maybe predict them, but the regression does not seem to be working'

In [ ]:
# quick ratio, cash ratio, net working capital ratio, curretn ratio vs liq ratio

#Tier 1 (core) risk-based capital
#Tier 2 risk-based capital
#Equity capital to assets
#Core capital (leverage) ratio

In [3]:
# commonstk = 3020000000
# toteqtycap = 236206000000
# riskwtd_assets = 1394763000000
# toteqtycap/riskwtd_assets * 100

16.93520691328921

## Basic Stats Analysis

In [5]:
df = pd.read_csv("../../data/raw/fin_statements/bofa_bs.csv")

In [7]:
df.head(10)
"""Last col -20 is all wrong, actually -grey cols"""

,Breakdown,Bank,12/31/2023,12/31/2022,12/31/2021,12/31/2020
0,Total Assets,BAC,"3,180,151,000","2,819,627,000","3,169,495,000","505,036,000"
1,"Cash, Cash Equivalents & Federal Funds Sold",BAC,"622,043,000","237,462,000","606,085,000","387,009,000"
2,Cash And Cash Equivalents,BAC,"341,419,000","36,430,000","355,365,000","199,869,000"
3,Cash,BAC,"27,892,000","7,259,000","29,222,000","6,546,000"
4,Cash And Due from Banks,BAC,"305,181,000","304,058,000","318,999,000","1,043,422,000"
5,Interest Bearing Deposits Assets,BAC,"8,346,000","180,603,000","7,144,000","107,344,000"
6,Money Market Investments,BAC,"280,624,000","12,110,000","250,720,000","632,825,000"
7,Securities and Investments,BAC,"1,017,946,000","220,788,000","1,126,273,000","234,491,000"
8,Trading Securities,BAC,"146,539,000","47,179,000","143,646,000","1,026,344,000"
9,Available for Sale Securities,BAC,"10,203,000","1,038,974,000","8,895,000","920,006,000"


### Liquidity

In [219]:
ca = ['Cash, Cash Equivalents & Federal Funds Sold',  'Available for Sale Securities',
      'Trading Securities', 'Other Short Term Investments', 'Loans Held for Sale', 'Accounts receivable', 
      'Other Receivables', 'Other Assets', 'Derivative Assets', 'Held To Maturity Securities']

# cce: 'Cash And Cash Equivalents', 'Cash', 'Cash And Due from Banks','Interest Bearing Deposits Assets', 'Money Market Investments',

 # this includes cash, cash eq, marketable securities, accs recievable
# I am not really seeing prepaid expenses or inventory on the balance sheet - they all seem quick enough to me
# or shld I split qa along trading securities, other short term inv, loans held for sale
qa = ['Cash, Cash Equivalents & Federal Funds Sold', 'Available for Sale Securities',
      'Trading Securities', 'Accounts receivable']



cl = ['Total Deposits','Federal Funds Purchased And Securities Sold Under Agreement To Repurchase', 'Current Accrued Expenses',
      'Current Debt And Capital Lease Obligation', 'Trading Liabilities', 'Derivative Product Liabilities', 'Other Liabilities']
# ambiguous = derviative assets, HMS, 


#### Quick ratio & Current Ratio

In [220]:
def get_qancl(df): # all rows of quick a and curr liab, ca for all banks
    quick_assets = []
    current_liabilities = []
    current_assets = []
    for idx, row in df.iterrows():
        if row['Breakdown'] in qa:
            quick_assets.append(row)
        if row["Breakdown"] in cl:
            current_liabilities.append(row)
        if row["Breakdown"] in ca:
            current_assets.append(row)
    return quick_assets, current_assets, current_liabilities
    
    # len(current_assets), len(current_liabilities)


In [221]:
def get_qancl_yr(yr, bank, quick_assets, current_assets, current_liabilities): # total canl for yr 
    # pass yr as index : '23-2, '22 - 3, '21-4, '20-5 
    qa_yr = 0
    cl_yr = 0
    ca_yr = 0
    for asset in quick_assets:
        if asset[1] == bank:
            print(asset[yr])
            val = int(asset[yr].replace(',', "").replace("-", "-1"))
            if val == -1:
                for i in range(2, 6):
                    if i != yr:
                        val += int(asset[i].replace(',', "")) 
                val = val/3
            qa_yr += val

    
    for lia in current_liabilities:
        if asset[1] == bank:
            val = int(lia[2].replace(',', "").replace("-", "-1"))
            if val == -1:
                for i in range(2, 6):
                    if i != yr:
                        val += int(asset[i].replace(',', ""))
                val = val/3
            cl_yr += val

    for asset in current_assets:
        if asset[1] == bank:
            val = int(asset[yr].replace(',', "").replace("-", "-1"))
            if val == -1:
                for i in range(2, 6):
                    if i != yr:
                        val += int(asset[i].replace(',', "")) 
                val = val/3
            ca_yr += val

    return qa_yr*1000, ca_yr*1000, cl_yr*1000  # for yr for bank
# all yf values are in thousands



In [222]:
quick_assets, current_assets, current_liabilities = get_qancl(df)
qa_23, ca_23, cl_23 = get_qancl_yr(4, 'BAC', quick_assets, current_assets, current_liabilities)
quick_ratio = qa_23/cl_23 
current_ratio = ca_23/cl_23

606,085,000
143,646,000
8,895,000
72,263,000


In [223]:
quick_ratio, qa_23/10**9, cl_23/10**9

(0.33441268393436324, 830.889, 2484.6216663333335)

In [224]:
current_ratio, ca_23/10**9

(0.4611172861946275, 1145.702)

In [225]:
# the grey cols are wrong - rescrape and check incom, cash flow also :===